# 数据库说明
data/zwx/qa目录系一个数据集，涵盖了包含中文问答对的 Excel 文件。在此文件中，第一列与第三列记录的是问题，第二列记录的是答案。第一列与第三列中的问题按多行进行划分，每行对应一个问题。

data/zwx/table目录包含了多个 Excel 文件，记录了表格数据。该文件的第一行是表头，后续行是数据内容。

In [1]:
# 从.env文件中加载环境变量（如API基础地址等）
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 导入所需库
import os
from typing import Dict, List
import requests

# 从环境变量中获取各服务的基础URL
EMBEDDING_GTE_QWEN_1_5B_INSTRUCT_BASE_URL = os.getenv("EMB_GTE_QWEN_1_5B_INSTRUCT_BASE_URL")
RERANKER_BGE_RERANKER_V2_M3_BASE_URL = os.getenv("BGE_RERANKER_V2_M3_BASE_URL")
LLM_QWEN2_5_7B_BASE_URL = os.getenv("LLM_QWEN2_5_7B_BASE_URL")

# 定义模型名称
LLM_QWEN2_5_7B_MODEL = "Qwen2.5-7B-Instruct"
BGE_RERANKER_V2_M3_MODEL = "bge-reranker-v2-m3"
EMBEDDING_GTE_QWEN_1_5B_INSTRUCT_MODEL = "gte_Qwen2-1.5B-instruct"

# 导入openai库
from openai import OpenAI
from openai.types import CreateEmbeddingResponse
import logging

# 初始化OpenAI客户端，用于LLM调用
client = OpenAI(base_url=LLM_QWEN2_5_7B_BASE_URL)

def get_completion(messages:List[Dict[str, str]], **kwargs) -> Dict[str, str]:
    """
    获取Qwen2.5-7B模型的文本生成结果
    :param messages: 消息列表，每个消息是一个字典，包含角色和内容
    :param kwargs: 可选参数，包括模型名称、API基础URL、API密钥、最大令牌数和温度等
    :return: 模型生成的文本结果
    """
    # 调用chat.completions.create方法获取模型回复
    return client.chat.completions.create(
        model=kwargs.get("model", LLM_QWEN2_5_7B_MODEL),
        messages=messages,
        max_tokens=kwargs.get("max_tokens", 8000),
        temperature=kwargs.get("temperature", 0),
    )

def get_embedding(text: str, **kwargs) -> CreateEmbeddingResponse:
    """
    获取文本的嵌入向量，返回CreateEmbeddingResponse类型
    :param text: 输入文本
    :param kwargs: 可选参数，包括模型名称、API基础URL和API密钥等
    :return: CreateEmbeddingResponse对象
    """
    url = f"{EMBEDDING_GTE_QWEN_1_5B_INSTRUCT_BASE_URL}/embeddings"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": kwargs.get("model", EMBEDDING_GTE_QWEN_1_5B_INSTRUCT_MODEL),
        "input": text
    }
    # 通过requests.post调用embedding服务
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    resp_json = response.json()
    # 构造CreateEmbeddingResponse对象
    return CreateEmbeddingResponse(**resp_json)

def get_rerank(query: str, documents: List[str], **kwargs) -> Dict[str, List[Dict[str, str]]]:
    """
    使用BGE Reranker对文档进行重排序
    :param query: 查询字符串
    :param documents: 文档列表，每个文档是字符串
    :param kwargs: 可选参数，包括模型名称、API基础URL和API密钥等
    :return: 包含重排序后的文档列表的字典
    """
    url = f"{RERANKER_BGE_RERANKER_V2_M3_BASE_URL}/rerank"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": kwargs.get("model", BGE_RERANKER_V2_M3_MODEL),
        "query": query,
        "documents": documents,
        "top_n": kwargs.get("top_n", 3)  # 默认返回前3个结果
    }
    # 调用重排序服务
    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [3]:
# 测试LLM，调用大模型进行对话
get_completion([
        {
            "role": "system",
            "content": "你是中卫信的疫苗接种助手“苗苗”，回答用户关于疫苗接种的相关问题。"
        },
        {
            "role": "user",
            "content": "你好"
        }
    ]
)

ChatCompletion(id='chatcmpl-b14db9fdabd149989d45e3a44e9fbb3f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！很高兴能为你提供关于疫苗接种的帮助。我是你的疫苗接种助手“苗苗”，请问你有关于疫苗接种方面的问题吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1748338982, model='Qwen2.5-7B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=34, total_tokens=65, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)

In [4]:
# 测试嵌入，获取文本的向量表示
get_embedding(
    "你好，苗苗",
    model=EMBEDDING_GTE_QWEN_1_5B_INSTRUCT_MODEL
)

CreateEmbeddingResponse(data=[Embedding(embedding=[0.00897979736328125, -0.01751708984375, 0.018096923828125, 0.007495880126953125, -0.01678466796875, -0.005245208740234375, -0.0015344619750976562, -0.0166473388671875, 0.02825927734375, 0.0155792236328125, -0.00577545166015625, 0.0367431640625, 0.024993896484375, 0.012451171875, -0.00653076171875, -0.0225372314453125, 0.00897216796875, 0.022369384765625, 0.041534423828125, 0.01334381103515625, 0.0035114288330078125, 0.0166015625, -0.00630950927734375, -0.0230712890625, -0.01007080078125, 0.0138397216796875, -0.0262908935546875, -0.0030193328857421875, 0.0029277801513671875, -0.0013637542724609375, 0.0032711029052734375, -0.027679443359375, -0.04638671875, 0.048126220703125, -0.034271240234375, 0.00959014892578125, -0.023223876953125, -0.00969696044921875, 0.021759033203125, -0.0064849853515625, 0.01541900634765625, 0.0211181640625, -0.01424407958984375, -0.010040283203125, 0.1214599609375, 0.018096923828125, -0.0087738037109375, -0.003

In [5]:
# 测试重排序，调用重排序服务对文档进行相关性排序
get_rerank(
    "旺庄街道服务中心可以接种犬苗吗？",
    documents=[
        "单位名称:通州区城区社区卫生服务中心，单位简称:城区社区卫生服务中心，单位地址:江苏省南通市通州区金新街道金霞路809号，儿童门诊联系电话:（0513）86029913或（0513）86029920，儿童门诊服务时间:每周一、三、五全天，周六上午（接种时间8:30-11:00，14:00-16:30）新冠接种除外，成人门诊联系电话:接种后如有不适请联系：（0513）86029913或（0513）86029920，成人门诊服务时间:周四，接种时间8:30-11:00，14:00-16:30(特殊情况周六上午08:30-11:00成人可接种)，服务范围:不可以打儿童疫苗，不可以打成人疫苗，可以打狂犬疫苗",
        "单位名称:旺庄街道社区卫生服务中心，单位简称:旺庄，单位地址:江苏省无锡市新吴区旺庄街道新光路316号新光路与锡士路交界，，儿童门诊联系电话:051085229130，儿童门诊服务时间:周二至周六，上午08:00-10:30（节假日除外）。，成人门诊联系电话:051085229130，成人门诊服务时间:周四13:30-15:00（节假日除外）自费疫苗接种服务费20元/针次，服务范围:不可以打儿童疫苗，可以打成人疫苗，可以打狂犬疫苗",   
        "单位名称:沭阳县沭城街道卫生健康服务中心，单位简称:沭城卫健中心（沭城接种门诊），单位地址:江苏省宿迁市沭阳县沭城街道中央商场西门对面.，儿童门诊联系电话:0527-82086255，儿童门诊服务时间:周一至周六，上午8：30-11:00，下午2:00-5:00  法定节假日除外，成人门诊联系电话:0527-82086255，成人门诊服务时间:周一至周六，上午8：30-11:00，下午2:00-5:00  法定节假日除外，狂犬门诊联系电话:0527-82086255，狂犬门诊服务时间:暂无狂犬疫苗接种，服务范围:不可以打儿童疫苗，可以打成人疫苗，可以打狂犬疫苗"        
    ]
)

{'id': 'rerank-84528df6098047fdb8c7cbd9f3d6f8b1',
 'model': 'bge-reranker-v2-m3',
 'usage': {'total_tokens': 569},
 'results': [{'index': 1,
   'document': {'text': '单位名称:旺庄街道社区卫生服务中心，单位简称:旺庄，单位地址:江苏省无锡市新吴区旺庄街道新光路316号新光路与锡士路交界，，儿童门诊联系电话:051085229130，儿童门诊服务时间:周二至周六，上午08:00-10:30（节假日除外）。，成人门诊联系电话:051085229130，成人门诊服务时间:周四13:30-15:00（节假日除外）自费疫苗接种服务费20元/针次，服务范围:不可以打儿童疫苗，可以打成人疫苗，可以打狂犬疫苗'},
   'relevance_score': 0.99609375},
  {'index': 2,
   'document': {'text': '单位名称:沭阳县沭城街道卫生健康服务中心，单位简称:沭城卫健中心（沭城接种门诊），单位地址:江苏省宿迁市沭阳县沭城街道中央商场西门对面.，儿童门诊联系电话:0527-82086255，儿童门诊服务时间:周一至周六，上午8：30-11:00，下午2:00-5:00  法定节假日除外，成人门诊联系电话:0527-82086255，成人门诊服务时间:周一至周六，上午8：30-11:00，下午2:00-5:00  法定节假日除外，狂犬门诊联系电话:0527-82086255，狂犬门诊服务时间:暂无狂犬疫苗接种，服务范围:不可以打儿童疫苗，可以打成人疫苗，可以打狂犬疫苗'},
   'relevance_score': 0.1044921875},
  {'index': 0,
   'document': {'text': '单位名称:通州区城区社区卫生服务中心，单位简称:城区社区卫生服务中心，单位地址:江苏省南通市通州区金新街道金霞路809号，儿童门诊联系电话:（0513）86029913或（0513）86029920，儿童门诊服务时间:每周一、三、五全天，周六上午（接种时间8:30-11:00，14:00-16:30）新冠接种除外，成人门诊联系电话:接种后如有不适

In [6]:
def embedding(text: str, **kwargs) -> List[float]:
    """
    获取文本的嵌入向量，返回一个浮点数列表
    :param text: 输入文本
    :param kwargs: 可选参数，包括模型名称、API基础URL和API密钥等
    :return: 嵌入向量列表
    """
    # 调用get_embedding获取嵌入结果
    response = get_embedding(text, **kwargs)
    # 返回第一个embedding结果（有些模型返回多条）
    # response.data[0].embedding 是一个浮点数列表，表示文本的高维向量
    return response.data[0].embedding if response.data else []

query_emb = embedding("你好，苗苗", model=EMBEDDING_GTE_QWEN_1_5B_INSTRUCT_MODEL)
import numpy as np
print(np.array(query_emb).astype('float32').reshape(1, -1))

[[ 0.0089798  -0.01751709  0.01809692 ...  0.006073   -0.02363586
   0.02011108]]


In [7]:
# 导入pandas和glob用于处理Excel文件
import pandas as pd
import glob
import os

# 问答对列表和知识库
qa_pairs = []

#  表格知识库文档
table_docs = []

# 如果问答对csv文件不存在，则从Excel文件生成
if not os.path.exists('data/zwx/qa_pairs.csv'):
    # 枚举目录下所有Excel文件
    excel_files = glob.glob(os.path.join('data/zwx/qa', '*.xls*'))

    for file in excel_files:
        # 读取Excel文件
        df = pd.read_excel(file)
        for idx, row in df.iterrows():
            # 第一列和第三列为问题，第二列为答案
            questions_col1 = str(row.iloc[0]).splitlines() if pd.notna(row.iloc[0]) else []
            answer = row.iloc[1] if pd.notna(row.iloc[1]) else ""
            questions_col3 = str(row.iloc[2]).splitlines() if pd.notna(row.iloc[2]) else []
            for q in questions_col1 + questions_col3:
                if q.strip():
                    qa_pairs.append({
                        # 问题
                        'question': q.strip(),
                        # 答案
                        'answer': answer,
                        # 获取问题的向量（高维浮点数列表，后续用于向量检索）
                        'embedding': embedding(q.strip()),
                        # 元数据，包含文件名和行索引
                        'metadata': {
                            'source': file,
                            'row_index': idx
                        }
                    })

    # 将问答对转换为DataFrame
    qa_df = pd.DataFrame(qa_pairs)
    # 保存为CSV文件
    qa_df.to_csv('data/zwx/qa_pairs.csv', index=False, encoding='utf-8-sig')

# 如果表格知识库csv文件不存在，则从Excel文件生成
if not os.path.exists('data/zwx/table.csv'):
    # 读取表格知识库
    table_files = glob.glob(os.path.join('data/zwx/table', '*.xls*'))
    for file in table_files:
        df = pd.read_excel(file)
        # 枚举每一行内容，并使用标题1:内容1, 标题2:内容2...拼装内容
        for idx, row in df.iterrows():
            row_content = "，".join([f"{col}:{row[col]}" for col in df.columns if pd.notna(row[col])])
            table_docs.append({
                'data': row_content,
                # 为每一行表格内容生成embedding向量（高维浮点数列表）
                'embedding': embedding(row_content),
                'metadata': {
                    'source': file,
                    'row_index': idx
                }
            })
    # 将表格知识库转换为DataFrame
    table_df = pd.DataFrame(table_docs)
    # 保存为CSV文件
    table_df.to_csv('data/zwx/table.csv', index=False, encoding='utf-8-sig')

In [8]:
# 导入faiss和numpy用于向量检索与存储
import faiss
import numpy as np
import pickle
import os
# 保存索引和qa_pairs到本地
os.makedirs('data/zwx/faiss', exist_ok=True)

# 把上面生成的问答对与向量存储到本地硬盘
if not os.path.exists('data/zwx/faiss/qa_pairs.pkl'):
    # 提取所有embedding向量，部分embedding可能是二维嵌套（兼容处理）
    # 每个embedding是一个高维浮点数列表，形如[0.1, 0.2, ...]
    embeddings = [
        item['embedding'][0] if isinstance(item['embedding'], list) and isinstance(item['embedding'][0], list)
        else item['embedding']
        for item in qa_pairs
    ]
    # 转为numpy数组，方便FAISS处理
    embeddings = np.array(embeddings).astype('float32')

    # 创建FAISS索引，L2距离（欧氏距离），dim为向量维度
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    # 将所有向量加入索引
    index.add(embeddings)

    # 保存索引到磁盘
    faiss.write_index(index, 'data/zwx/faiss/qa.index')
    # 保存问答对及其元数据到磁盘
    with open('data/zwx/faiss/qa_pairs.pkl', 'wb') as f:
        pickle.dump(qa_pairs, f)

# 保存表格知识库到本地
if not os.path.exists('data/zwx/faiss/table_docs.pkl'):
    # 提取所有表格数据的向量
    table_embeddings = [doc['embedding'] for doc in table_docs]
    # 转为numpy数组
    table_embeddings = np.array(table_embeddings).astype('float32')

    # 创建FAISS索引
    dim = table_embeddings.shape[1]
    table_index = faiss.IndexFlatL2(dim)
    table_index.add(table_embeddings)

    # 保存表格索引和数据
    faiss.write_index(table_index, 'data/zwx/faiss/table.index')
    with open('data/zwx/faiss/table_docs.pkl', 'wb') as f:
        pickle.dump(table_docs, f)

In [9]:
# 导入faiss和pickle用于加载本地索引和数据
import faiss
import numpy as np
import pickle

# 加载FAISS索引和qa_pairs
index = faiss.read_index('data/zwx/faiss/qa.index')
with open('data/zwx/faiss/qa_pairs.pkl', 'rb') as f:
    qa_pairs = pickle.load(f)

def search_qa(query, top_k=3, min_similarity=0.5):
    """
    使用FAISS索引检索最相似的问答对
    :param query: 查询字符串
    :param top_k: 返回的最相似问答对数量
    :param min_similarity: 最小相似度阈值
    :return: 包含最相似问答对的列表
    """
    # 获取查询的embedding（高维浮点数列表）
    query_emb = embedding(query)
    # 兼容部分模型返回二维嵌套的情况
    if isinstance(query_emb, list) and isinstance(query_emb[0], list):
        query_emb = np.array(query_emb[0]).astype('float32').reshape(1, -1)
    else:
        query_emb = np.array(query_emb).astype('float32').reshape(1, -1)
    # 使用FAISS索引进行向量检索，返回距离最近的top_k*3条（便于后续去重）
    # D为距离，I为索引
    D, I = index.search(query_emb, top_k * 3)
    results = []
    seen_answers = set()
    for dist, idx in zip(D[0], I[0]):
        if idx < 0 or idx >= len(qa_pairs):
            continue
        # FAISS L2距离越小越相似，这里用sim = 1 / (1 + dist)近似转为相似度
        sim = 1 / (1 + dist)
        qa_item = qa_pairs[idx].copy()  # 深拷贝，避免修改原数据
        qa_item["similarity"] = float(sim)  # 添加相似度字段
        qa_item["index"] = int(idx)
        # 删除embedding字段，避免输出过大
        del qa_item["embedding"]
        answer = qa_item['answer']
        # 答案去重且相似度大于阈值才加入结果
        if sim >= min_similarity and answer not in seen_answers:
            results.append(qa_item)
            seen_answers.add(answer)
        if len(results) >= top_k:
            break
    # 按相似度降序排序
    results.sort(key=lambda x: x['similarity'], reverse=True)
    return results

# 加载表格知识库
with open('data/zwx/faiss/table_docs.pkl', 'rb') as f:
    table_docs = pickle.load(f)

def search_table(query, top_k=3, min_similarity=0.2):
    """
    使用FAISS索引检索最相似的表格数据
    :param query: 查询字符串
    :param top_k: 返回的最相似表格数据数量
    :param min_similarity: 最小相似度阈值
    :return: 包含最相似表格数据的列表
    """
    # 获取查询的embedding（高维浮点数列表）
    query_emb = embedding(query)
    # 兼容部分模型返回二维嵌套的情况
    if isinstance(query_emb, list) and isinstance(query_emb[0], list):
        query_emb = np.array(query_emb[0]).astype('float32').reshape(1, -1)
    else:
        query_emb = np.array(query_emb).astype('float32').reshape(1, -1)
    # 读取表格知识库的FAISS索引
    table_index = faiss.read_index('data/zwx/faiss/table.index')
    # 检索最相似的top_k*3条
    D, I = table_index.search(query_emb, top_k * 3)
    results = []
    seen_data = set()
    for dist, idx in zip(D[0], I[0]):
        if idx < 0 or idx >= len(table_docs):
            continue
        # FAISS L2距离越小越相似，这里用sim = 1 / (1 + dist)近似转为相似度
        sim = 1 / (1 + dist)
        table_item = table_docs[idx]  # 直接取表格项
        data = table_item['data']
        answer = {
            "answer": data,
            "similarity": float(sim),
            "metadata": table_item['metadata'],
            "index": int(idx)
        }
        # 数据去重且相似度大于阈值才加入结果
        if sim >= min_similarity and data not in seen_data:
            results.append(answer)
            seen_data.add(data)
        if len(results) >= top_k:
            break
    # 按相似度降序排序
    results.sort(key=lambda x: x['similarity'], reverse=True)
    return results

In [10]:
# 示例：查询，检索最相关的问答对
search_qa("3岁儿童可以打百白破吗？", top_k=3)

[{'question': '没有接种过百白破疫苗的人，需要补接种百白破疫苗吗？',
  'answer': '6岁及以下儿童：可根据疫苗间隔要求完成百白破疫苗的补种。\n6岁以上儿童及成人：如果百白破和白破疫苗共计接种未满3剂次，应使用白破疫苗补齐至3剂，确保充分保护。',
  'metadata': {'source': 'data/zwx/qa/接种知识.xlsx', 'row_index': 119},
  'similarity': 0.7472346425056458,
  'index': 1247},
 {'question': '百白破疫苗应该在什么年龄接种？接种几个剂次？',
  'answer': '自2025年1月1日起，按2月龄、4月龄、6月龄、18月龄、6周岁各接种1剂次百白破疫苗的免疫程序接种。',
  'metadata': {'source': 'data/zwx/qa/接种知识.xlsx', 'row_index': 213},
  'similarity': 0.718840479850769,
  'index': 1433},
 {'question': '未按时接种百白破疫苗，需要补种吗？',
  'answer': '未按时接种百白破疫苗应根据当地卫生部门的指导进行补种，通常建议尽早补种以确保充分保护。',
  'metadata': {'source': 'data/zwx/qa/接种知识.xlsx', 'row_index': 117},
  'similarity': 0.7082926630973816,
  'index': 1243}]

In [11]:
# 测试表格查询，检索最相关的表格数据
search_table("旺庄街道服务中心可以接种犬苗吗？", top_k=3)

[{'answer': '单位名称:通州区城区社区卫生服务中心，单位简称:城区社区卫生服务中心，单位地址:江苏省南通市通州区金新街道金霞路809号，儿童门诊联系电话:（0513）86029913或（0513）86029920，儿童门诊服务时间:每周一、三、五全天，周六上午（接种时间8:30-11:00，14:00-16:30）新冠接种除外，成人门诊联系电话:接种后如有不适请联系：（0513）86029913或（0513）86029920，成人门诊服务时间:周四，接种时间8:30-11:00，14:00-16:30(特殊情况周六上午08:30-11:00成人可接种)，服务范围:不可以打儿童疫苗，不可以打成人疫苗，可以打狂犬疫苗',
  'similarity': 0.6022469401359558,
  'metadata': {'source': 'data/zwx/table/江苏省门诊单位信息.xlsx', 'row_index': 942},
  'index': 942},
 {'answer': '单位名称:旺庄街道社区卫生服务中心，单位简称:旺庄，单位地址:江苏省无锡市新吴区旺庄街道新光路316号新光路与锡士路交界，，儿童门诊联系电话:051085229130，儿童门诊服务时间:周二至周六，上午08:00-10:30（节假日除外）。，成人门诊联系电话:051085229130，成人门诊服务时间:周四13:30-15:00（节假日除外）自费疫苗接种服务费20元/针次，服务范围:不可以打儿童疫苗，可以打成人疫苗，可以打狂犬疫苗',
  'similarity': 0.6022008061408997,
  'metadata': {'source': 'data/zwx/table/江苏省门诊单位信息.xlsx', 'row_index': 250},
  'index': 250},
 {'answer': '单位名称:沭阳县沭城街道卫生健康服务中心，单位简称:沭城卫健中心（沭城接种门诊），单位地址:江苏省宿迁市沭阳县沭城街道中央商场西门对面.，儿童门诊联系电话:0527-82086255，儿童门诊服务时间:周一至周六，上午8：30-11:00，下午2:00-5:00  法定节假日除外，成人门诊联系电话:0527-82086255，成人门诊服务时间:周一

In [12]:
from openai import OpenAI

def llm_reply(query, top_k=3):
    """
    使用LLM回复用户问题
    :param query: 用户问题
    :param top_k: 检索的最相似问答对数量
    :return: LLM的回复内容
    """
    # 先通过qa搜索
    answers = search_qa(query, top_k=top_k, min_similarity=0.6)

    # 如果没有找到相关答案，尝试通过表格知识库搜索
    if not answers:
        table_answers = search_table(query, top_k=top_k, min_similarity=0.4)
        if table_answers:
            answers = table_answers

    if not answers:
        answers = ["抱歉，我不知道答案。"]

    answer_texts = [i["answer"] for i in answers if "answer" in i]
    reranked_results = get_rerank(query, answer_texts, top_n=1).get("results", [])
    
    answers = [result.get("document", {}).get("text") for result in reranked_results]

    # 拼接检索到的答案作为上下文
    context = "\n".join(answers)
    # 构造llm输入
    prompt = f"已知信息：\n{context}\n\n用户问题：{query}\n请从已知信息中提取原文回复。如果不知道答案，请礼貌地告诉用户您不知道。"
    # 调用llm接口
    response = get_completion([
        {
            "role": "system",
            "content": "你是中卫信的疫苗接种助手“苗苗”，回答用户关于疫苗接种的相关问题。"
        },
        {
            "role": "user",
            "content": prompt
        }
    ])
    return response.choices[0].message.content

# 示例：调用llm_reply进行问答
llm_reply("请问乙肝疫苗多少钱？", top_k=3)

'乙肝CHO疫苗的单价区间为60.00元至90.00元；乙肝酿酒酵母疫苗的单价区间为35.00元至230.00元；乙肝汉逊酵母疫苗的单价区间为13.50元至88.00元。\n\n请注意，不同地区使用的疫苗品种、批次、厂家可能不同，因此价格可能有所差异。此外，非免疫规划类疫苗接种通常会收取20元服务费。'

In [13]:
# 示例：直接调用llm_reply
llm_reply("你好")

'您可以微信关注育苗通公众号进行留言，或者在工作日时间08:30-12:00、13:30-17:30拨打客服热线4001132929。'

data/zwx/test目录中存放了测试数据，其中第一列是问题

In [ ]:
import os
import glob
import logging
import pandas as pd

# 设置日志输出
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')


# 枚举data/zwx/test目录下所有Excel文件
test_files = glob.glob(os.path.join('data/zwx/test', '*.xls*'))

for file in test_files:
    print(f"正在处理文件: {file}")
    df = pd.read_excel(file)
    # 假设第一列为问题
    replies = []
    for idx, row in df.iterrows():
        question = str(row.iloc[0]).strip()
        if question:
            answer = llm_reply(question)
        else:
            answer = ""
        replies.append(answer)
    # 添加新列
    df["模型回复"] = replies
    # 保存到新文件
    base, ext = os.path.splitext(file)
    result_file = base + "_result" + ext
    df.to_excel(result_file, index=False)
    print(f"已保存: {result_file}")

正在处理文件: data/zwx/test/AI100.xlsx


2025-05-27 17:43:12,437 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:13,270 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:14,406 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:15,695 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:16,542 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:19,002 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:19,317 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:20,900 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-27 17:43:21,368 INFO HTTP Request: POST http://10.88.99.116:7001/v1/chat/completions "HTTP/1.1 200 OK"
2